In [ ]:
import subprocess
import numpy as np
from pyfinite import ffield
np.set_printoptions(threshold=np.inf)


In [ ]:
# Develop a mapping
mapping = {}
for i in range(16):
    num = '{:0>4}'.format(format(i, "b"))
    numi = int(num[3]) + 2 * int(num[2]) + int(num[1]) * 4 + int(num[0])*8
    mapping[num] = chr(ord('f')+numi)


In [ ]:
#Create 128*8 inputs file from f to u for known plaintext attack
file = open("inputs.txt", "w+")
for i in range(8):
    for j in range(128):
        curr_ip_j = np.binary_repr(j, width=8)
        strr = 'ff'*i + mapping[curr_ip_j[:4]] + \
            mapping[curr_ip_j[4:]] + 'ff'*(8-i-1)
        file.write(strr)
        file.write(" ")
    file.write("\n")
file.close()

In [ ]:
# Create a similar bash input file to interact with the shell
file = open("bash_input.txt", "w+")
file.write("3Miners1Course\n")
file.write("cryptoiscool\n")
file.write("5\n")
file.write("go\n")
file.write("wave\n")
file.write("dive\n")
file.write("go\n")
file.write("read\n")
for i in range(8):
    for j in range(128):
        curr_ip_j = np.binary_repr(j, width=8)
        strr = 'ff'*i + mapping[curr_ip_j[:4]] + \
            mapping[curr_ip_j[4:]] + 'ff'*(8-i-1)
        file.write(strr)
        file.write("\n")
        file.write("c\n")
file.write("back\n")
file.write("exit")
file.close()


In [ ]:
#Run Script file to generate outputs for corrersponding bash inputs
subprocess.run("bash script.sh",shell=True)

In [ ]:
# Parse bash outputs to allot dimensions
file = open('bash_output.txt', mode='r', encoding='utf-8-sig')
line = file.readlines()
text = []
for i in line:
    i = i.strip()
    text.append(i)
file.close()
file = open("outputs.txt", "w+")
for i in range(1024):
    if i != 0 and i % 128 == 0:
        file.write("\n")
    file.write(text[i])
    file.write(" ")
file.close()


In [ ]:
# Create a 0-16 map-set
map_set = {'0000': 'f',
           '0001': 'g',
           '0010': 'h',
           '0011': 'i',
           '0100': 'j',
           '0101': 'k',
           '0110': 'l',
           '0111': 'm',
           '1000': 'n',
           '1001': 'o',
           '1010': 'p',
           '1011': 'q',
           '1100': 'r',
           '1101': 's',
           '1110': 't',
           '1111': 'u'}


In [ ]:
# Utility functions
exp_arr = [[-1]*128 for i in range(128)]

# Defining a Field
field = ffield.FField(7)


def add(n1, n2):
    return int(n1) ^ int(n2)


def multiply(n1, n2):
    return field.Multiply(n1, n2)


def exp(no, pow):
    if exp_arr[no][pow] != -1:
        return exp_arr[no][pow]

    ans = 0
    if pow == 0:
        ans = 1
    elif pow == 1:
        ans = no
    elif pow % 2 == 0:
        sqrt_no = exp(no, pow >> 1)
        ans = multiply(sqrt_no, sqrt_no)
    else:
        sqrt_no = exp(no, pow >> 1)
        ans = multiply(sqrt_no, sqrt_no)
        ans = multiply(no, ans)

    exp_arr[no][pow] = ans
    return ans


def add_vectors(v1, v2):
    ans = [0]*8
    for i, (e1, e2) in enumerate(zip(v1, v2)):
        ans[i] = add(e1, e2)
    return ans


def scalar_mul(v, elem):
    ans = [0]*8
    for i, e in enumerate(v):
        ans[i] = multiply(e, elem)
    return ans


def lin_transform(mat, elist):
    ans = [0]*8
    for row, elem in zip(mat, elist):
        ans = add_vectors(scalar_mul(row, elem), ans)
    return ans

def encrypt(plaintext, lin_mat, exp_mat):
    plaintext = [ord(c) for c in plaintext]
    ans = [[0 for j in range (8)] for i in range(8)]
    for idx, elem in enumerate(plaintext):
        ans[0][idx] = exp(elem, exp_mat[idx])

    ans[1] = lin_transform(lin_mat, ans[0])

    for idx, elem in enumerate(ans[1]):
        ans[2][idx] = exp(elem, exp_mat[idx])

    ans[3] = lin_transform(lin_mat, ans[2])

    for idx, elem in enumerate(ans[3]):
        ans[4][idx] = exp(elem, exp_mat[idx])
    return ans[4]


# Map byte to string(2 chars)
def byte_to_str(b):
    binnum = '{:0>8}'.format(format(b, "b"))
    a = map_set[binnum[0:4]], map_set[binnum[4:8]]
    return a[0]+a[1]

# Map byte to hex
def byte_to_hex(st):
    char = chr(16*(ord(st[0]) - ord('f')) + ord(st[1]) - ord('f'))
    return char

# Map block to hex
def block_to_hex(c):
    plainText = ""
    for i in range(0, len(c), 2):
        plainText += byte_to_hex(c[i:i+2])
    return plainText


In [ ]:
#Finding possible values for diagonal and exponent matrices
possible_exp = [[] for i in range(8)]
possible_diag = [[[] for i in range(8)] for j in range(8)]

input_file = open("inputs.txt", 'r')
output_file = open("outputs.txt", 'r')

for idx, (input_line, output_line) in enumerate(zip(input_file.readlines(), output_file.readlines())):
    input_str = []
    output_str = []
    for hex_input in input_line.strip().split(" "):
        input_str.append(block_to_hex(hex_input)[idx])
    for hex_output in output_line.strip().split(" "):
        output_str.append(block_to_hex(hex_output)[idx])
    for i in range(1, 127):
        for j in range(1, 128):
            flag = True
            for inps, outs in zip(input_str, output_str):
                if ord(outs) != exp(multiply(exp(multiply(exp(ord(inps), i), j), i), j), i):
                    flag = False
                    break
            if flag:
                possible_exp[idx].append(i)
                possible_diag[idx][idx].append(j)
print(possible_diag)
print(possible_exp)


In [ ]:
# Similarly eliminate extra values

input_file = open("inputs.txt", 'r')
output_file = open("outputs.txt", 'r')

for idx, (input_line, output_line) in enumerate(zip(input_file.readlines(), output_file.readlines())):
    if idx > 6 :
        break
    input_str = []
    output_str = []
    for hex_input in input_line.strip().split(" "):
        input_str.append(block_to_hex(hex_input)[idx]) 
    for hex_output in output_line.strip().split(" "):
        output_str.append(block_to_hex(hex_output)[idx+1])
    for i in range(1, 128):
        for p1, e1 in zip(possible_exp[idx+1], possible_diag[idx+1][idx+1]):
            for p2, e2 in zip(possible_exp[idx], possible_diag[idx][idx]):
                flag = True
                for inp, outp in zip(input_str, output_str):
                    if ord(outp) != exp(add(multiply(exp(multiply(exp(ord(inp), p2), e2), p2), i) ,multiply(exp(multiply(exp(ord(inp), p2), i), p1), e1)), p1):
                        flag = False
                        break
                if flag:
                    possible_exp[idx+1] = [p1]
                    possible_diag[idx+1][idx+1] = [e1]
                    possible_exp[idx] = [p2]
                    possible_diag[idx][idx] = [e2]
                    possible_diag[idx][idx+1] = [i]
print(possible_diag)
print(possible_exp)

In [ ]:
for idx in range(6):
    of = idx + 2
    
    exp_list = [e[0] for e in possible_exp]
    lin_trans_list = [[0 for i in range(8)] for j in range(8)]
    for i in range(8):
        for j in range(8):
            lin_trans_list[i][j] = 0 if len(possible_diag[i][j]) == 0 else possible_diag[i][j][0]
    input_file = open("inputs.txt", 'r')
    output_file = open("outputs.txt", 'r')
    for idx, (input_line, output_line) in enumerate(zip(input_file.readlines(), output_file.readlines())):
        if idx > (7-of):
            continue
        input_str = [block_to_hex(msg) for msg in input_line.strip().split(" ")]
        output_str = [block_to_hex(msg) for msg in output_line.strip().split(" ")]
        for i in range(1, 128):
            lin_trans_list[idx][idx+of] = i
            flag = True
            for inps, outs in zip(input_str, output_str):
                if encrypt(inps, lin_trans_list, exp_list)[idx+of] != ord(outs[idx+of]):
                    flag = False
                    break
            if flag:
                possible_diag[idx][idx+of] = [i]
    input_file.close()
    output_file.close()
# Fill all the empty [] elements with 0
lin_trans_list = [[0 for i in range(8)] for j in range(8)]
for i in range(8):
    for j in range(8):
        lin_trans_list[i][j] = 0 if len(possible_diag[i][j]) == 0 else possible_diag[i][j][0]

print(lin_trans_list)
print(exp_list)

In [ ]:
# final values obtained from above:
linear_mat = [[84, 125, 18, 105, 100, 24, 11, 76],
                 [0, 70, 28, 22, 43, 39, 118, 8],
                 [0, 0, 43, 12, 1, 19, 8, 95],
                 [0, 0, 0, 12, 122, 46, 101, 27],
                 [0, 0, 0, 0, 112, 101, 22, 24],
                 [0, 0, 0, 0, 0, 11, 94, 67],
                 [0, 0, 0, 0, 0, 0, 27, 11],
                 [0, 0, 0, 0, 0, 0, 0, 38]]
exponent_mat = [17, 114, 40, 68, 90, 40, 25, 15]

encrypted_password = "imhgjgfqgjlqlimghohjlugullkifkhs"
password_1 = "imhgjgfqgjlqlimg"
password_2 = "hohjlugullkifkhs"

# Brute force to perform encrypt
def decrypt_pswd(pswd):
    pswd_hex = block_to_hex(pswd)
    decrypted_pswd = ""
    for idx in range(8):
        for ans in range(128):
            inp = decrypted_pswd + byte_to_str(ans)+(16-len(decrypted_pswd)-2)*'f'
            if ord(pswd_hex[idx]) == encrypt(block_to_hex(inp), linear_mat, exponent_mat)[idx]:
                decrypted_pswd += byte_to_str(ans)
                break
    return decrypted_pswd

final_pswd = block_to_hex(decrypt_pswd(password_1)) + block_to_hex(decrypt_pswd(password_2))
print("Password is:", final_pswd.strip("0"))
